In [ ]:
import matplotlib.pyplot as plt
import osmnx as ox

In [ ]:
places = {
    "Dongcheng": "Dongcheng, Beijing, China",
    "Xicheng": "Xicheng, Beijing, China",
    "Chaoyang": "Chaoyang, Beijing, China",
    "Fengtai": "Fengtai, Beijing, China",
    "Shijingshan": "Shijingshan, Beijing, China",
    "Haidian": "Haidian, Beijing, China",
    "Mentougou": "Mentougou, Beijing, China",
    "Fangshan": "Fangshan, Beijing, China",
    "Daxing": "Daxing, Beijing, China",
    "Tongzhou": "Tongzhou, Beijing, China",
    "Shunyi": "Shunyi, Beijing, China",
    "Changping": "Changping, Beijing, China",
    "Huairou": "Huairou, Beijing, China",
    "Pinggu": "Pinggu, Beijing, China",
    "Miyun": "Miyun, Beijing, China",
    "Yanqing": "Yanqing, Beijing, China"
} # Beijing

In [ ]:
places = {
    "Xuanwu": "Xuanwu, Nanjing, China",
    "Qinhuai": "Qinhuai, Nanjing, China",
    "Jianye": "Jianye, Nanjing, China",
    "Gulou": "Gulou, Nanjing, China",
    "Qixia": "Qixia, Nanjing, China",
    "Yuhuatai": "Yuhuatai, Nanjing, China",
    "Jiangning": "Jiangning, Nanjing, China",
    "Pukou": "Pukou, Nanjing, China",
    "Luhe": "Luhe, Nanjing, China",
    "Lishui": "Lishui, Nanjing, China",
    "Gaochun": "Gaochun, Nanjing, China",
    "Jiangbei": "Jiangbei, Nanjing, China",
} # Nanjing

In [ ]:
gdf = ox.geocoder.geocode_to_gdf(list(places.values()))
gdf

In [ ]:
gdf.plot()

In [ ]:
# fetch the street network for each city
street_graphs = {}
for place in places:
    print(ox.utils.ts(), place)
    street_graphs[place] = ox.graph.graph_from_place(places[place], network_type="drive")


In [ ]:
import math
def primary_angle(angle):
    if angle > 180:
        return angle - 180
    elif angle < 0:
        return angle + 180
    return angle

def calc_entropy(angles, bins=18):
    counts = [0 for _ in range(bins)]
    for angle in angles:
        a = int(angle-0.0001) // (180 // bins)
        counts[a] += 1
    # print('counts', counts)

    entropy = 0
    for i, cnt in enumerate(counts):
        entropy -= cnt/sum(counts) * math.log(cnt / sum(counts))
    return entropy


In [ ]:
plt.get_cmap('plasma')

In [ ]:
n = len(places)
ncols = 4
nrows = (len(places)+3)//ncols
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={"projection": "polar"})

# plot each city's polar histogram
Gus = []
for i, place in enumerate(places):
    ax = axes.flat[i]
    G = street_graphs[place]
    Gu = ox.bearing.add_edge_bearings(ox.convert.to_undirected(G))

    angles = [primary_angle(d['bearing']) for _, _, _, d in Gu.edges(keys=True, data=True) if 'bearing' in d]

    Gus.append((calc_entropy(angles), place, Gu))

Gus = sorted(Gus, key=lambda x: x[0], reverse=True)
# norm = plt.Normalize(2, 3)
norm = plt.Normalize(Gus[-1][0], Gus[0][1])
cmap = plt.get_cmap('plasma')
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
for i, (entropy, place, gu) in enumerate(Gus):
    ax = axes.flat[nrows*ncols-i-1]
    fig, ax = ox.plot.plot_orientation(gu, ax=ax, title=place, area=True, color=cmap(norm(entropy)))

# add figure title and save image
suptitle_font = {
    "family": "DejaVu Sans",
    "fontsize": 60,
    "fontweight": "normal",
    "y": 1,
}
fig.suptitle("Street Network Orientation", **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))

gdf.plot(ax=ax, color="none", edgecolor='k', linewidth=1.2, zorder=10)

for i, (entropy, place, _) in enumerate(Gus):
    color = cmap(norm(entropy))
    gdf_edges = ox.convert.graph_to_gdfs(street_graphs[place], nodes=False)["geometry"]
    gdf_edges.plot(ax=ax, color=color, linewidth=0.5, alpha=0.5)
    gdf_nodes = ox.convert.graph_to_gdfs(street_graphs[place], edges=False, node_geometry=False)[["x", "y"]]
    ax.scatter(x=gdf_nodes["x"], y=gdf_nodes["y"], color=color, zorder=2, s=0.5)

fig.tight_layout()
fig.subplots_adjust(hspace=0.35)

sm.set_array([])
plt.rcParams.update({
    'font.size': 20,
    'font.family': 'DejaVu Sans',
    'font.weight': 'normal',
})
cbar = fig.colorbar(sm, ax=ax, location='left')
cbar.set_label("Orientation Entropy", fontsize=30, fontweight='bold')
plt.axis("off")
fig.savefig("orientation_entropy.png", bbox_inches='tight')
